In [127]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from sklearn.model_selection import KFold
import lightgbm as lgb
from sklearn.metrics import accuracy_score

In [128]:
train = pd.read_csv('PJT002_train.csv')
val = pd.read_csv('PJT002_validation.csv')
test = pd.read_csv('PJT002_test.csv')

sub = pd.read_csv('PJT002_submission.csv')
data = pd.concat([train, val])

In [129]:
a = train[train['fr_yn'] == 'Y']
b = train[train['fr_yn'] == 'N']


In [130]:
a.describe()

,bldng_cnt,bldng_ar,ttl_ar,lnd_ar,ttl_grnd_flr,ttl_dwn_flr,tmprtr,prcpttn,wnd_spd,wnd_drctn,...,fr_wthr_fclt_dstnc,fr_mn_cnt,cctv_dstnc,fr_wthr_fclt_in_100m,cctv_in_100m,tbc_rtl_str_dstnc,sft_emrgnc_bll_dstnc,ahsm_dstnc,no_tbc_zn_dstnc,bldng_cnt_in_50m
count,7657.000000,7657.000000,7.657000e+03,7.657000e+03,7222.000000,7116.000000,7657.000000,510.000000,7654.000000,7624.000000,...,7657.000000,7643.000000,7657.000000,7657.000000,7657.000000,7657.000000,7657.000000,7657.000000,7657.000000,7657.000000
mean,4.506334,2669.058081,1.466057e+04,1.691188e+05,8.232761,0.363547,14.130547,2.313922,1.836922,176.625787,...,778.605720,153.648044,658.596709,0.398459,0.499543,2683.234426,31477.614209,12352.868486,943.413217,4.328066
std,13.797839,23424.764105,2.164877e+05,5.706383e+06,32.667197,1.832651,10.001980,6.178839,1.410417,121.568109,...,1343.318966,39.892599,1268.047719,0.762676,1.425357,3088.968003,24372.224516,13306.951776,1648.441868,9.091445
min,1.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,-14.100000,0.000000,0.000000,0.000000,...,0.000000,85.000000,2.000000,0.000000,0.000000,0.000000,8.000000,0.000000,0.000000,0.000000
25%,1.000000,88.600000,9.630000e+01,0.000000e+00,1.000000,0.000000,5.800000,0.000000,0.800000,70.000000,...,74.000000,112.000000,87.000000,0.000000,0.000000,396.000000,11862.000000,1108.000000,42.000000,0.000000
50%,2.000000,163.980000,2.309300e+02,3.280000e+02,2.000000,0.000000,14.600000,0.500000,1.500000,200.000000,...,192.000000,161.000000,216.000000,0.000000,0.000000,1395.000000,29487.000000,7280.000000,231.000000,0.000000
75%,4.000000,522.240000,9.931600e+02,9.220000e+02,4.000000,0.000000,22.300000,2.000000,2.600000,290.000000,...,871.000000,194.000000,703.000000,1.000000,0.000000,3950.000000,45074.000000,20711.000000,1031.000000,4.000000
max,321.000000,572897.955000,1.235184e+07,2.038984e+08,413.000000,36.000000,37.900000,93.000000,15.300000,360.000000,...,10709.000000,210.000000,12166.000000,7.000000,24.000000,18185.000000,100256.000000,56485.000000,11944.000000,75.000000


In [131]:
b.describe()

,bldng_cnt,bldng_ar,ttl_ar,lnd_ar,ttl_grnd_flr,ttl_dwn_flr,tmprtr,prcpttn,wnd_spd,wnd_drctn,...,fr_wthr_fclt_dstnc,fr_mn_cnt,cctv_dstnc,fr_wthr_fclt_in_100m,cctv_in_100m,tbc_rtl_str_dstnc,sft_emrgnc_bll_dstnc,ahsm_dstnc,no_tbc_zn_dstnc,bldng_cnt_in_50m
count,51542.000000,51542.000000,51542.000000,5.154200e+04,41767.000000,41078.000000,51529.000000,5060.000000,51512.000000,51354.000000,...,51542.000000,51515.000000,51542.000000,51542.000000,51542.000000,51542.000000,51542.000000,51542.000000,51542.000000,51542.000000
mean,3.297583,331.172692,668.127928,8.184471e+03,2.321737,0.092434,13.998556,1.662569,1.490596,157.245122,...,2078.109057,150.858973,775.173334,0.223488,0.402992,3337.980463,31383.927923,8472.370106,1176.641128,3.739009
std,5.605187,3904.918898,7625.981560,7.106883e+05,11.046060,0.764534,9.753160,3.693185,1.274548,126.170125,...,2740.338640,36.262461,1346.419359,0.578866,1.134385,3356.095577,24230.789912,10974.433271,1672.467743,8.301792
min,1.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,-13.200000,0.000000,0.000000,0.000000,...,0.000000,85.000000,0.000000,0.000000,0.000000,0.000000,5.000000,0.000000,0.000000,0.000000
25%,1.000000,40.000000,46.280000,0.000000e+00,1.000000,0.000000,6.100000,0.000000,0.500000,20.000000,...,126.000000,112.000000,102.000000,0.000000,0.000000,553.000000,11175.000000,733.000000,131.000000,0.000000
50%,2.000000,82.500000,90.360000,1.361500e+02,1.000000,0.000000,14.800000,0.400000,1.100000,160.000000,...,780.000000,161.000000,301.000000,0.000000,0.000000,2307.000000,26614.000000,3007.000000,453.000000,0.000000
75%,4.000000,134.797500,185.120000,3.999750e+02,2.000000,0.000000,21.900000,1.500000,2.100000,270.000000,...,2918.000000,176.000000,912.000000,0.000000,0.000000,5073.000000,45854.000000,13005.000000,1546.000000,3.000000
max,705.000000,515929.687000,640508.141000,1.196523e+08,594.000000,60.000000,38.100000,48.600000,13.900000,360.000000,...,17716.000000,210.000000,12278.000000,10.000000,24.000000,23228.000000,103571.000000,56169.000000,16164.000000,84.000000


In [121]:
train['dt_of_athrztn'].fillna(9999,inplace  = True)

In [122]:
train['dt_of_athrztn'] = train['dt_of_athrztn'].astype(str,inplace = True)

In [123]:
train['dt_of_athrztn']=train['dt_of_athrztn'].str[:4]

In [124]:
train['dt_of_athrztn'] = train['dt_of_athrztn'].astype(int,inplace = True)

In [125]:

bins = np.linspace(1600,2019, 20)
train['dt_of_athrztn'] = np.digitize(train['dt_of_athrztn'], bins)


In [126]:
val['dt_of_athrztn'].fillna(9999,inplace  = True)

val['dt_of_athrztn'] = val['dt_of_athrztn'].astype(str,inplace = True)

val['dt_of_athrztn']=val['dt_of_athrztn'].str[:4]

val['dt_of_athrztn'] = val['dt_of_athrztn'].astype(int,inplace = True)

len(val['dt_of_athrztn'].unique())

bins = np.linspace(1600,2019, 20)
val['dt_of_athrztn'] = np.digitize(val['dt_of_athrztn'], bins)

test['dt_of_athrztn'].fillna(9999,inplace  = True)

test['dt_of_athrztn'] = test['dt_of_athrztn'].astype(str,inplace = True)

test['dt_of_athrztn']=test['dt_of_athrztn'].str[:4]

test['dt_of_athrztn'] = test['dt_of_athrztn'].astype(int,inplace = True)

len(test['dt_of_athrztn'].unique())
bins = np.linspace(1600,2019, 20)
test['dt_of_athrztn'] = np.digitize(test['dt_of_athrztn'], bins)

In [92]:
test = test.drop(['fr_yn'],1)

In [93]:
data['dt_of_fr'] = pd.to_datetime(data['dt_of_fr'])
train['dt_of_fr'] = pd.to_datetime(train['dt_of_fr'])
val['dt_of_fr'] = pd.to_datetime(val['dt_of_fr'])
test['dt_of_fr'] = pd.to_datetime(test['dt_of_fr'])



In [94]:
categorical_cols1 = train.select_dtypes(['object']).columns
for col in categorical_cols1:
    train[col] = pd.Categorical(train[col]).codes


train['DT_hour'] = (train['dt_of_fr'].dt.hour)
train['DT_day_week'] = (train['dt_of_fr'].dt.dayofweek)
train['DT_day_month'] = (train['dt_of_fr'].dt.day)
train['DT_M'] = (train['dt_of_fr'].dt.month)
train['DT_Y'] = (train['dt_of_fr'].dt.year)
train = train.drop('dt_of_fr', axis = 1)

In [95]:
categorical_cols1 = val.select_dtypes(['object']).columns
for col in categorical_cols1:
    val[col] = pd.Categorical(val[col]).codes


val['DT_hour'] = (val['dt_of_fr'].dt.hour)
val['DT_day_week'] = (val['dt_of_fr'].dt.dayofweek)
val['DT_day_month'] = (val['dt_of_fr'].dt.day)
val['DT_M'] = (val['dt_of_fr'].dt.month)
val['DT_Y'] = (val['dt_of_fr'].dt.year)
val = val.drop('dt_of_fr', axis = 1)

In [96]:
categorical_cols1 = test.select_dtypes(['object']).columns
for col in categorical_cols1:
    test[col] = pd.Categorical(test[col]).codes


test['DT_hour'] = (test['dt_of_fr'].dt.hour)
test['DT_day_week'] = (test['dt_of_fr'].dt.dayofweek)
test['DT_day_month'] = (test['dt_of_fr'].dt.day)
test['DT_M'] = (test['dt_of_fr'].dt.month)
test['DT_Y'] = (test['dt_of_fr'].dt.year)
test = test.drop('dt_of_fr', axis = 1)

In [118]:
a = train[train['fr_yn'] == 0]

a['hmdt'].mean()

67.11909937888198

In [117]:
a = train[train['fr_yn'] == 1]

a['hmdt'].mean()

57.29202037351443

In [ ]:
from tqdm import tqdm_notebook
for i in tqdm_notebook((4,5,6,7,8)):
    for j in tqdm_notebook((10,11,12)):
        #bins = np.linspace(0, 1000000, 1000)
        train['agas_engry_us_201%s%s'%(i,j)] = train['gas_engry_us_201%s%s'%(i,j)] / train['ttl_ar']
        #bins = np.linspace(0, 1000000, 1000)
        val['agas_engry_us_201%s%s'%(i,j)] = val['gas_engry_us_201%s%s'%(i,j)]/ train['ttl_ar']
        #bins = np.linspace(0, 1000000, 1000)
        test['agas_engry_us_201%s%s'%(i,j)] =  test['gas_engry_us_201%s%s'%(i,j)]/ train['ttl_ar']
        

from tqdm import tqdm_notebook
for i in tqdm_notebook((4,5,6,7,8)):
    for j in tqdm_notebook((1,2,3,4,5,6,7,8,9)):
        #bins = np.linspace(0, 1000000, 1000)
        train['agas_engry_us_201%s%s'%(i,j)] = train['gas_engry_us_201%s0%s'%(i,j)] / train['ttl_ar']
        #bins = np.linspace(0, 1000000, 1000)
        val['agas_engry_us_201%s%s'%(i,j)] = val['gas_engry_us_201%s0%s'%(i,j)]/ train['ttl_ar']
        #bins = np.linspace(0, 1000000, 1000)
        test['agas_engry_us_201%s%s'%(i,j)] =  test['gas_engry_us_201%s0%s'%(i,j)]/ train['ttl_ar']

from tqdm import tqdm_notebook
for i in tqdm_notebook((4,5,6,7,8)):
    for j in tqdm_notebook((10,11,12)):
        #bins = np.linspace(0, 1000000, 1000)
        train['aele_engry_us_201%s%s'%(i,j)] = train['ele_engry_us_201%s%s'%(i,j)]/ train['ttl_ar']
        #bins = np.linspace(0, 1000000, 1000)
        val['aele_engry_us_201%s%s'%(i,j)] = val['ele_engry_us_201%s%s'%(i,j)]/ train['ttl_ar']
        #bins = np.linspace(0, 1000000, 1000)
        test['aele_engry_us_201%s%s'%(i,j)] = test['ele_engry_us_201%s%s'%(i,j)]/ train['ttl_ar']
        

from tqdm import tqdm_notebook
for i in tqdm_notebook((4,5,6,7,8)):
    for j in tqdm_notebook((1,2,3,4,5,6,7,8,9)):
        #bins = np.linspace(0, 1000000, 1000)
        train['aele_engry_us_201%s%s'%(i,j)] = train['ele_engry_us_201%s0%s'%(i,j)]/ train['ttl_ar']
        #bins = np.linspace(0, 1000000, 1000)
        val['aele_engry_us_201%s%s'%(i,j)] = val['ele_engry_us_201%s0%s'%(i,j)]/ train['ttl_ar']
        #bins = np.linspace(0, 1000000, 1000)
        test['aele_engry_us_201%s%s'%(i,j)] = test['ele_engry_us_201%s0%s'%(i,j)]/ train['ttl_ar']
        


In [ ]:
from tqdm import tqdm_notebook
for i in tqdm_notebook((4,5,6,7,8)):
    for j in tqdm_notebook((10,11,12)):
        bins = np.linspace(0, 1000000, 50000)
        train['tagas_engry_us_201%s%s'%(i,j)] = np.digitize(train['gas_engry_us_201%s%s'%(i,j)], bins)
        bins = np.linspace(0, 1000000, 50000)
        val['tagas_engry_us_201%s%s'%(i,j)] = np.digitize(val['gas_engry_us_201%s%s'%(i,j)], bins)
        bins = np.linspace(0, 1000000, 50000)
        test['tagas_engry_us_201%s%s'%(i,j)] = np.digitize(test['gas_engry_us_201%s%s'%(i,j)], bins)
        

from tqdm import tqdm_notebook
for i in tqdm_notebook((4,5,6,7,8)):
    for j in tqdm_notebook((1,2,3,4,5,6,7,8,9)):
        bins = np.linspace(0, 1000000, 50000)
        train['tagas_engry_us_201%s0%s'%(i,j)] = np.digitize(train['gas_engry_us_201%s0%s'%(i,j)], bins)
        bins = np.linspace(0, 1000000, 50000)
        val['tagas_engry_us_201%s0%s'%(i,j)] = np.digitize(val['gas_engry_us_201%s0%s'%(i,j)], bins)
        bins = np.linspace(0, 1000000, 50000)
        test['tagas_engry_us_201%s0%s'%(i,j)] = np.digitize(test['gas_engry_us_201%s0%s'%(i,j)], bins)

from tqdm import tqdm_notebook
for i in tqdm_notebook((4,5,6,7,8)):
    for j in tqdm_notebook((10,11,12)):
        bins = np.linspace(0, 1000000, 50000)
        train['taele_engry_us_201%s%s'%(i,j)] = np.digitize(train['ele_engry_us_201%s%s'%(i,j)], bins)
        bins = np.linspace(0, 1000000, 50000)
        val['taele_engry_us_201%s%s'%(i,j)] = np.digitize(val['ele_engry_us_201%s%s'%(i,j)], bins)
        bins = np.linspace(0, 1000000, 50000)
        test['taele_engry_us_201%s%s'%(i,j)] = np.digitize(test['ele_engry_us_201%s%s'%(i,j)], bins)
        

train['gassum'] = 0
val['gassum'] = 0
test['gassum'] = 0

from tqdm import tqdm_notebook
for i in tqdm_notebook((4,5,6,7,8)):
    for j in tqdm_notebook((10,11,12)):
        train['gassum'] = train['gassum'] + train['gas_engry_us_201%s%s'%(i,j)]
        val['gassum'] = val['gassum'] + val['gas_engry_us_201%s%s'%(i,j)]
        test['gassum'] = test['gassum'] + test['gas_engry_us_201%s%s'%(i,j)]
       
for i in tqdm_notebook((4,5,6,7,8)):
    for j in tqdm_notebook((1,2,3,4,5,6,7,8,9)):
        train['gassum'] = train['gassum'] + train['gas_engry_us_201%s0%s'%(i,j)]
        val['gassum'] = val['gassum'] + val['gas_engry_us_201%s0%s'%(i,j)]
        test['gassum'] = test['gassum'] + test['gas_engry_us_201%s0%s'%(i,j)]

from tqdm import tqdm_notebook
for i in tqdm_notebook((4,5,6,7,8)):
    for j in tqdm_notebook((1,2,3,4,5,6,7,8,9)):
        bins = np.linspace(0, 1000000, 50000)
        train['taele_engry_us_201%s0%s'%(i,j)] = np.digitize(train['ele_engry_us_201%s0%s'%(i,j)], bins)
        bins = np.linspace(0, 1000000, 50000)
        val['taele_engry_us_201%s0%s'%(i,j)] = np.digitize(val['ele_engry_us_201%s0%s'%(i,j)], bins)
        bins = np.linspace(0, 1000000, 50000)
        test['taele_engry_us_201%s0%s'%(i,j)] = np.digitize(test['ele_engry_us_201%s0%s'%(i,j)], bins)



train['elesum'] = 0
val['elesum'] = 0
test['elesum'] = 0

from tqdm import tqdm_notebook
for i in tqdm_notebook((4,5,6,7,8)):
    for j in tqdm_notebook((10,11,12)):
        train['elesum'] = train['elesum'] + train['ele_engry_us_201%s%s'%(i,j)]
        val['elesum'] = val['elesum'] + val['ele_engry_us_201%s%s'%(i,j)]
        test['elesum'] = test['elesum'] + test['ele_engry_us_201%s%s'%(i,j)]
       
for i in tqdm_notebook((4,5,6,7,8)):
    for j in tqdm_notebook((1,2,3,4,5,6,7,8,9)):
        train['elesum'] = train['elesum'] + train['ele_engry_us_201%s0%s'%(i,j)]
        val['elesum'] = val['elesum'] + val['ele_engry_us_201%s0%s'%(i,j)]
        test['elesum'] = test['elesum'] + test['ele_engry_us_201%s0%s'%(i,j)]

for i in tqdm_notebook((4,5,6,7,8)):
    for j in tqdm_notebook((1,2,3,4,5,6,7,8,9)):       
        train = train.drop(['ele_engry_us_201%s0%s'%(i,j)],1)
        train = train.drop(['gas_engry_us_201%s0%s'%(i,j)],1)


        val = val.drop(['ele_engry_us_201%s0%s'%(i,j)],1)
        val = val.drop(['gas_engry_us_201%s0%s'%(i,j)],1)


        test = test.drop(['ele_engry_us_201%s0%s'%(i,j)],1)
        test = test.drop(['gas_engry_us_201%s0%s'%(i,j)],1)
for i in tqdm_notebook((4,5,6,7,8)):
    for j in tqdm_notebook((10,11,12)):       
        train = train.drop(['ele_engry_us_201%s%s'%(i,j)],1)
        train = train.drop(['gas_engry_us_201%s%s'%(i,j)],1)


        val = val.drop(['ele_engry_us_201%s%s'%(i,j)],1)
        val = val.drop(['gas_engry_us_201%s%s'%(i,j)],1)


        test = test.drop(['ele_engry_us_201%s%s'%(i,j)],1)
        test = test.drop(['gas_engry_us_201%s%s'%(i,j)],1)

In [72]:
bins = np.linspace(0,800000, 1000)
train['ttl_ar'] = np.digitize(train['ttl_ar'], bins)
bins = np.linspace(0,800000, 1000)
val['ttl_ar'] = np.digitize(val['ttl_ar'], bins)
bins = np.linspace(0,800000, 1000)
test['ttl_ar'] = np.digitize(test['ttl_ar'], bins)


In [73]:
X_train = train.drop(['fr_yn'],axis = 1)
X_test = val.drop(['fr_yn'],axis = 1)
y_train = train['fr_yn']
y_test = val['fr_yn']
X = pd.concat([X_train,X_test])
y = pd.concat([y_train,y_test])

In [74]:

X_train.drop(['id'], axis = 1, inplace = True)
test.drop(['id'], axis = 1, inplace = True)
X_test.drop(['id'], axis = 1, inplace = True)

In [76]:
#모델 생성 및 학습
for i in range(300):
    
    import lightgbm as lgb
    Lgb = lgb.LGBMClassifier(n_estimators=i+1, 
                             silent=False, 
                             random_state =10, 
                             max_depth=18,
                             num_leaves=208,
                             learning_rate = 0.1,
                             objective='binary',
                             metrics ='auc'
                            )
    model = Lgb.fit(X_train, y_train)

#test를 통한 모델 검증
    predictions = model.predict(X_test)
    proba = model.predict_proba(X_test)
    from sklearn.metrics import accuracy_score, roc_auc_score
    from sklearn.metrics import f1_score, recall_score
    score = roc_auc_score(y_test, predictions)
    score2 = accuracy_score(y_test, predictions)
    score3 = f1_score(y_test, predictions)
    score4 = recall_score(y_test, predictions)
#     print('roc_auc_score:',score)
#     print('accuracy_score:',score2)
    print(i+1,":",'F1', score3)
    #print('recall', score4)

1 : F1 0.0
2 : F1 0.0
3 : F1 0.0
4 : F1 0.0
5 : F1 0.0031397174254317113
6 : F1 0.28028814669286184
7 : F1 0.33851897946484133
8 : F1 0.3720930232558139
9 : F1 0.41202185792349727
10 : F1 0.43612108773730124
11 : F1 0.4622093023255814
12 : F1 0.47601304145319046
13 : F1 0.4863883847549909
14 : F1 0.48487544483985767
15 : F1 0.4908616187989556
16 : F1 0.49723286504895703
17 : F1 0.49564857024450887
18 : F1 0.498793242156074
19 : F1 0.4968203497615262
20 : F1 0.5021543282412847
21 : F1 0.5049961568024597
22 : F1 0.5053353658536586
23 : F1 0.5045180722891567
24 : F1 0.5043347154165096
25 : F1 0.5027974636329727
26 : F1 0.500554528650647
27 : F1 0.4998181156784285
28 : F1 0.4998189061934082
29 : F1 0.4998207242739333
30 : F1 0.4998217468805704
31 : F1 0.4987513378523011
32 : F1 0.5003548616039745


KeyboardInterrupt: 

In [56]:
#34
y = model.predict(test)
y_prob = model.predict_proba(test)


In [57]:
sub['fr_yn'] = y
sub['fr_yn'] = sub['fr_yn'].map({0:'N', 1:'Y'})
sub.head()

,fr_yn
0,N
1,N
2,N
3,N
4,N


In [58]:
sub.to_csv('tmddnjs94_화재예측과제_진우31파라미터.csv', index=False)

In [ ]:
a['tf']
a['tf'] = pd.Categorical(a['tf']).codes

In [ ]:
sub['fr_yn'] = sub['fr_yn'].map({0:'N', 1:'Y'})

In [ ]:
sub.info()